<figure>
  <IMG SRC="Logo.png"  WIDTH=150 ALIGN="right">
</figure>

## Projections of mean sea level and tides<br>
### Paper in Journal of Coastal and Hydraulic Structures
<b>Prepared by: H.G. Voortman</b>


<figure>
    <IMG SRC="https://www.python.org/static/community_logos/python-powered-w-200x80.png"  WIDTH=100 ALIGN="right">
</figure>

#### Description
- Analyses described in section 3 of the paper

#### Packages

In [42]:
# General packages
import os
import copy as cp
import sqlite3 as sq
import pandas as pd
import numpy as np
import hvec_tide as tide
from matplotlib import pyplot as plt

# Company packages
from hvec_support import sqlite as hvsq
import hvec_tide as tide

# Project package
from utils import names


In [43]:
tide.__version__

'3.4.0'

#### Import data

In [44]:
os.chdir(os.getenv('DATAPATH'))
cnxn = hvsq.connect_verbose(r'RWS_data.db', detect_types = True)

The database  RWS_data.db  is opened.
This database contains the following tables: 
                 name
0       RWS_Waterinfo
1  Logboek_databeheer
2        Download_log


In [45]:
sql = 'SELECT DISTINCT(naam) FROM RWS_Waterinfo'
names = pd.read_sql(sql, cnxn)

In [46]:
names[names['naam'].str.contains('muid')]

,naam
8,IJmuiden Noordersluis O
9,IJmuiden Buitenhaven
10,IJmuiden Noordersluis W
11,IJmuiden buitenhaven
12,IJmuiden stroommeetpaal
13,IJmuiden Noordersluis
14,IJmuiden binnen
15,IJmuiden zuidelijk havenhoofd
16,IJmuiden semafoor
39,IJmuiden Zuid havenhoofd


In [47]:
sql = (
    'SELECT * FROM RWS_Waterinfo WHERE '
    'naam IN ("Harlingen", "Hoek van Holland", "IJmuiden Noordersluis", "Den Helder", "Harlingen", "Delfzijl", "Vlissingen") '
    'AND year IN (2020) '
    )
df = pd.read_sql(sql, cnxn)

In [48]:
settings = {
    'nameColumn': 'naam'
    , 'timeColumn': 'tijd'
    , 'levelColumn': 'waarde'
}

In [49]:
coef = pd.DataFrame(df.groupby(['naam', 'year']).apply(lambda x: tide.analysers.select_constituents(x, latitude = 52, settings = settings, thr = 90))).reset_index()

In [50]:
coef.rename(
    columns = {0: 'constit'}, inplace = True
)

In [51]:
coef

,naam,year,constit
0,Delfzijl,2020,"[M2, S2]"
1,Den Helder,2020,"[M2, S2, M4, SA, N2, O1]"
2,Harlingen,2020,"[M2, S2, N2, SA]"
3,Hoek van Holland,2020,"[M2, S2, M4, N2]"
4,IJmuiden Noordersluis,2020,"[M2, M4, S2, MS4, O1, N2]"
5,Vlissingen,2020,[M2]


#### End script
Prepared by HVEC lab, 2022